In [1]:
# Reference code from Session 13 - Heaps and priority queues

import heapq
class Task:
    """
    - id: Task id (a reference number)   
    - description: Task short description   
    - duration: Task duration in minutes   
    - dependencies: List of task ids that need to preceed this task  
    - status: Current status of the task     
   
    """
    #Initializes an instance of Task
    def __init__(self, id, description, duration,
                 dependencies, status="N"):
        self.id = id
        self.description = description
        self.duration = duration
        self.dependencies = dependencies
        self.status = status

    def __lt__(self, other):
        return self.id < other.id    
    
class TaskScheduler:
    """
    A Simple Daily Task Scheduler Using Priority Queues
    (that doesn't really work!)
    """
    NOT_STARTED = 'N'
    IN_PRIORITY_QUEUE = 'I'
    COMPLETED = 'C'
    
    def __init__(self, tasks):
        self.tasks = tasks
        self.priority_queue = []     
        
    def print_self(self):
        print("Tasks added to the simple scheduler:")
        print("--------------------------------------")
        for t in self.tasks:
            print(f"➡️'{t.description}', duration = {t.duration} mins.")   
            if len(t.dependencies)>0:
                print(f"\t ⚠️ This task depends on others!")     
            
    def remove_dependency(self, id):
        for t in self.tasks:
            if t.id != id and id in t.dependencies:
                t.dependencies.remove(id)           
            
    def get_tasks_ready(self):
        for task in self.tasks:
            # If the task has no dependencies and is not yet in the queue
            if task.status == self.NOT_STARTED and not task.dependencies: 
                # Update status of the task
                task.status = self.IN_PRIORITY_QUEUE 
                # Push task into the priority queue
                ####### YOUR CODE HERE (1 line of code required)
                heapq.heappush(self.priority_queue, task)
    
    def check_unscheduled_tasks(self):
        for task in self.tasks:
            if task.status == self.NOT_STARTED:
                return True
        return False   
    
    def format_time(self, time):
        return f"{time//60}h{time%60:02d}"
    #🕰t=8h00
    
    def run_task_scheduler(self, starting_time):
        current_time = starting_time
        print("Running a simple scheduler:\n")
        while self.check_unscheduled_tasks() or self.priority_queue:
            # STEPs 1 and 2: 
            # Identify tasks that are ready to execute 
            # (those without dependencies) and 
            # push them into the priority queue
            self.get_tasks_ready()
            #STEP 3: Check for tasks in the priority queue
            if len(self.priority_queue) > 0 :      
                # STEP 4: get the task on top of the priority queue 
                ####### YOUR CODE HERE (1 line of code required)
                task = heapq.heappop(self.priority_queue)
                print(f"🕰t={self.format_time(current_time)}")
                print(f"\tstarted '{task.description}' for {task.duration} mins...")
                current_time += task.duration            
                print(f"\t✅ t={self.format_time(current_time)}, task completed!") 
                # If the task is done, remove it from the dependency list 
                ####### YOUR CODE HERE (1 line of code required)
                self.remove_dependency(task.id)
                task.status = self.COMPLETED
        total_time = current_time - starting_time             
        print(f"\n🏁 Completed all planned tasks in {total_time//60}h{total_time%60:02d}min!")

In [2]:
# Reference code from Session 13 - Heaps and priority queues
tasks = [
    Task(id=0, description='get up at 8:00 AM', 
         duration=10, dependencies=[]), 
    Task(id=1, description='get dressed and ready', 
         duration=10, dependencies=[0]), 
    Task(id=2, description='eat healthy breakfast', 
         duration=40, dependencies=[0]), 
    Task(id=3, description='make grocery list', 
         duration=20, dependencies=[0]), 
    Task(id=4, description='go to the market', 
         duration=15, dependencies=[1, 3]), 
    Task(id=5, description='buy groceries in the list', 
         duration=30, dependencies=[4]), 
    Task(id=6, description='drive back home', 
         duration=15, dependencies=[5]), 
    Task(id=7, description='store groceries', 
         duration=5, dependencies=[6])
    ]

task_scheduler = TaskScheduler(tasks)

# print the scheduler's input 
task_scheduler.print_self()

Tasks added to the simple scheduler:
--------------------------------------
➡️'get up at 8:00 AM', duration = 10 mins.
➡️'get dressed and ready', duration = 10 mins.
	 ⚠️ This task depends on others!
➡️'eat healthy breakfast', duration = 40 mins.
	 ⚠️ This task depends on others!
➡️'make grocery list', duration = 20 mins.
	 ⚠️ This task depends on others!
➡️'go to the market', duration = 15 mins.
	 ⚠️ This task depends on others!
➡️'buy groceries in the list', duration = 30 mins.
	 ⚠️ This task depends on others!
➡️'drive back home', duration = 15 mins.
	 ⚠️ This task depends on others!
➡️'store groceries', duration = 5 mins.
	 ⚠️ This task depends on others!


In [3]:
# Reference code from Session 13 - Heaps and priority queues
start_scheduler_at = 8*60
task_scheduler.run_task_scheduler(start_scheduler_at)

Running a simple scheduler:

🕰t=8h00
	started 'get up at 8:00 AM' for 10 mins...
	✅ t=8h10, task completed!
🕰t=8h10
	started 'get dressed and ready' for 10 mins...
	✅ t=8h20, task completed!
🕰t=8h20
	started 'eat healthy breakfast' for 40 mins...
	✅ t=9h00, task completed!
🕰t=9h00
	started 'make grocery list' for 20 mins...
	✅ t=9h20, task completed!
🕰t=9h20
	started 'go to the market' for 15 mins...
	✅ t=9h35, task completed!
🕰t=9h35
	started 'buy groceries in the list' for 30 mins...
	✅ t=10h05, task completed!
🕰t=10h05
	started 'drive back home' for 15 mins...
	✅ t=10h20, task completed!
🕰t=10h20
	started 'store groceries' for 5 mins...
	✅ t=10h25, task completed!

🏁 Completed all planned tasks in 2h25min!
